In [5]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, Input, Output, dash_table
import plotly.express as px
import pandas as pd
from crud import AnimalShelter

# Initialize CRUD module
username = "aacuser"
password = "cs340"
shelter = AnimalShelter(username, password)

# Initialize Dash application
app = JupyterDash(__name__)

# App layout
app.layout = html.Div([
    html.Img(
        src='/assets/Grazioso Salvare Logo.png',  # logo image path and display settings
        style={
            'height': '150px',
            'width': '300px',
            'margin': 'auto',
            'display': 'block'
        }
    ),
    html.Center(html.B(html.H1('Grazioso Salvare Rescue Dashboard - Ty Wheeler'))),

    
    dcc.RadioItems( # radio filter selections
        id='rescue-filter',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain Rescue', 'value': 'mountain'},
            {'label': 'Disaster Rescue', 'value': 'disaster'},
            {'label': 'Reset Filter', 'value': 'reset'}
        ],
        value='reset',
        style={'padding': '20px'}
    ),
    
    # Data table for displaying animal records
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in shelter.read({})[0].keys() if i != '_id'],
        page_action="native",
        page_current=0,
        page_size=10,
        style_table={'overflowX': 'auto'}
    ),
    
    html.Br(),
    html.Hr(),
    
    # Visualization container with two columns
    html.Div([
        dcc.Graph(id='geo-chart', className='col s6'), # Map visualization
        dcc.Graph(id='outcome-chart', className='col s6') # Outcome distribution
    ], style={'display': 'flex'})
])

# Define callback to update dashboard components
@app.callback(
    [Output('datatable-id', 'data'),
     Output('geo-chart', 'figure'),
     Output('outcome-chart', 'figure')],
    [Input('rescue-filter', 'value')]
)
def update_dashboard(rescue_type): # Determine query based on rescue type
    if rescue_type == 'reset':
        query = {}
    else:
        query = shelter.get_rescue_query(rescue_type)
    
    # Get filtered data
    data = shelter.read(query)
    df = pd.DataFrame(data).drop(columns=['_id'], errors='ignore')

    # Handle empty dataset
    if df.empty:
        return [], px.scatter_mapbox().update_layout(title="No Data Available"), px.bar().update_layout(title="No Outcomes Available")

    # Ensure that necessary columns exist
    for col in ['location_lat', 'location_long', 'outcome_type']:
        if col not in df.columns:
            df[col] = None  # Prevent any missing column errors

    # Update Data Table
    table_data = df.to_dict('records')

    # Update Map
    geo_fig = px.scatter_mapbox(
        df,
        lat='location_lat',
        lon='location_long',
        hover_name='breed',
        zoom=10,
        height=400
    )
    geo_fig.update_layout(mapbox_style="open-street-map")

    # Update Outcome type graph
    if df['outcome_type'].notna().sum() > 0:  # Check if there are valid values
        outcome_counts = df['outcome_type'].value_counts().reset_index()
        outcome_fig = px.bar(
            outcome_counts,
            x='index',
            y='outcome_type',
            labels={'index': 'Outcome Type', 'outcome_type': 'Count'},
            title='Outcome Types'
        )
    else:
        outcome_fig = px.bar().update_layout(title="No Outcomes Available")

    return table_data, geo_fig, outcome_fig

# Start the application
app.run_server(debug=True)

Dash app running on http://127.0.0.1:10444/
